# Eat Safe, Love

## Notebook Set Up

In [2]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} documents with a hygiene score of 20.")

# Display the first document in the results using pprint
results = establishments.find(query)
print("")
print("The first document is:")
print("-"*50)
pprint(results[0])

There are 41 documents with a hygiene score of 20.

The first document is:
--------------------------------------------------
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('674b75e0f59df7a019d19611'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api

In [8]:
# Convert the result to a Pandas DataFrame
hygiene_score = pd.json_normalize(results)
hygiene_score.head()

# Display the number of rows in the DataFrame
print(f"The number of rows in the hygiene_score DataFrame is: {len(hygiene_score)}")
print("")

# Display the first 10 rows of the DataFrame
hygiene_score.head(10)

# Convert Dataframe to CSV file
hygiene_score.to_csv("Outputs/hygiene_score.csv", index=False, header=True)


The number of rows in the hygiene_score DataFrame is: 41



### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [9]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName":{'$regex': "London"}, "RatingValue" : {'$gte':4}}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} documents for establishments within 'London' with a 'RatingValue' >= 4.")

# Display the first document in the results using pprint
results = establishments.find(query)
print("")
print("The first document is:")
print("-"*50)
pprint(results[0])

There are 33 documents for establishments within 'London' with a 'RatingValue' >= 4.

The first document is:
--------------------------------------------------
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('674b75e1f59df7a019d1b02c'),
 'ge

In [10]:
# REFERENCE: https://www.geeksforgeeks.org/normalizing-nested-json-object-into-pandas-dataframe/?ref=header_outind

# Convert the result to a Pandas DataFrame
london_rating_value = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(london_rating_value)}")

# Display the first 10 rows of the DataFrame
london_rating_value.head(10)

# Convert Dataframe to CSV file
london_rating_value.to_csv("Outputs/london_rating_value.csv", index=False, header=True)

The number of rows in this DataFrame is: 33


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [11]:
query = {"BusinessName": "Penang Flavours"}
fields = {"geocode.latitude": 1, "geocode.longitude": 1, "_id": 0}

# Execute the query
penang_flavours = establishments.find_one(query, fields)

# Print the latitude and longitude
if penang_flavours:
    latitude = penang_flavours['geocode']['latitude']
    longitude = penang_flavours['geocode']['longitude']
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("Penang Flavours not found in the collection.")

Latitude: 51.490142, Longitude: 0.08384


In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("hygiene", 1)]
limit = 5

results = list(establishments.find(query).sort(sort).limit(limit))


# Print the results
for document in results:
    pprint(document)

{'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Howe and Co Fish and Chips - Van 17',
 'BusinessType': 'Mobile caterer',
 'BusinessTypeID': 7846,
 'ChangesByServerID': 0,
 'Distance': 4646.95593107927,
 'FHRSID': 1380578,
 'LocalAuthorityBusinessID': '14425',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1SE',
 'RatingDate': '2021-11-11T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('674b75e3f59df7a019d1ee53'),
 'geocode': {'latitude': 51.4875335693359, 'longitude': 0.0925370007753372},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1380578',
            'rel': 'self'}],


In [13]:
# Convert result to Pandas DataFrame
penang_geo_proximity = pd.json_normalize(results)
penang_geo_proximity

# Convert Dataframe to CSV file
penang_geo_proximity.to_csv("Outputs/penang_geo_proximity.csv", index=False, header=True)



### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match = {'$match': {"scores.Hygiene":0}}

# 2. Groups the matches by Local Authority
group = {'$group': {"_id" : "$LocalAuthorityName",
                   "count": {'$sum':1}}}

# 3. Sorts the matches from highest to lowest
sort = {'$sort':{"count":-1}}

# 4. Pipeline aggregation
pipeline = [match, group, sort]

# 5. Results
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
print("")
print("The first 10 documents are:")
print("-"*50)
pprint(results[0:10])

There are 55 documents in the result.

The first 10 documents are:
--------------------------------------------------
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [15]:
# Convert the result to a Pandas DataFrame
pipeline_DF = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(pipeline_DF)}")
print("")

# Display the first 10 rows of the DataFrame
pipeline_DF.head(10)

# Convert Dataframe to CSV file
pipeline_DF.to_csv("Outputs/pipeline_DF.csv", index=False, header=True)

The number of rows in this DataFrame is: 55

